# L2: Build Your First Semantic Cache

In this lab, you’ll build a working semantic cache from scratch so you can see how each part works, and then you will implement it using Redis’s open source SDK and database.


In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install -r requirements.txt

## Load the FAQ Dataset

In [ ]:
import pandas as pd
import numpy as np
import time

from cache.faq_data_container import FAQDataContainer

faq_data = FAQDataContainer()
faq_df = faq_data.faq_df
test_df = faq_data.test_df

: 

In [ ]:
faq_df.head().style

: 

In [10]:
!pip install -U sentence-transformers huggingface_hub transformers

  Using cached sentence_transformers-5.1.2-py3-none-any.whl.metadata (16 kB)
  Using cached huggingface_hub-1.1.5-py3-none-any.whl.metadata (13 kB)
  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
Using cached sentence_transformers-5.1.2-py3-none-any.whl (488 kB)
Using cached transformers-4.57.1-py3-none-any.whl (12.0 MB)
Using cached huggingface_hub-0.36.0-py3-none-any.whl (566 kB)
Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl (2.7 MB)

  Attempting uninstall: huggingface_hub

    Found existing installation: huggingface-hub 0.34.4

    Uninstalling huggingface-hub-0.34.4:

      Successfully uninstalled huggingface-hub-0.34.4

   ---------------------------------------- 0/4 [huggingface_hub]
   ---------------------------------------- 0/4 [huggingface_hub]
   ---------------------------------------- 0/4 [huggin

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.19.1 requires fsspec[http]<=2024.3.1,>=2023.1.0, but you have fsspec 2024.6.1 which is incompatible.

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Create Embeddings for Semantic Search

In [ ]:
from sentence_transformers import SentenceTransformer

encoder = SentenceTransformer("all-mpnet-base-v2")

faq_embeddings = encoder.encode(faq_df["question"].tolist())

print(f"Sample (first 10 dimensions): {faq_embeddings[0][:10]}")

ModuleNotFoundError: Could not import module 'PreTrainedModel'. Are this object's requirements defined correctly?

## Implement Semantic Search

In [ ]:
def cosine_dist(a: np.array, b: np.array):
    """Compute cosine distance between two sets of vectors."""
    a_norm = np.linalg.norm(a, axis=1)
    b_norm = np.linalg.norm(b) if b.ndim == 1 else np.linalg.norm(b, axis=1)
    sim = np.dot(a, b) / (a_norm * b_norm)
    return 1 - sim


def semantic_search(query: str) -> tuple:
    """Find the most similar FAQ question to the query."""
    query_embedding = encoder.encode([query])[0]

    distances = cosine_dist(faq_embeddings, query_embedding)

    # Find the most similar question (lowest distance)
    best_idx = int(np.argmin(distances))
    best_distance = distances[best_idx]

    return best_idx, best_distance

In [ ]:
idx, distance = semantic_search(
    "How long will it take to get a refund for my order?"
)

print(f"Most similar FAQ: {faq_df.iloc[idx]['question']}")
print(f"Answer: {faq_df.iloc[idx]['answer']}")
print(f"Cosine distance: {distance:.3f}")

## Build a Simple Semantic Cache

In [ ]:
def check_cache(query: str, distance_threshold: float = 0.3):
    """
    Semantic cache lookup for previously asked questions.
    Returns a dictionary with answer if hit, None if miss.
    """
    idx, distance = semantic_search(query)

    if distance <= distance_threshold:
        return {
            "prompt": faq_df.iloc[idx]["question"],
            "response": faq_df.iloc[idx]["answer"],
            "vector_distance": float(distance),
        }

    return None  # Cache miss

In [ ]:
test_queries = [
    "Is it possible to get a refund?",
    "I want my money back",
    "What are your business hours?",  # Should miss
]

for query in test_queries:
    result = check_cache(query, distance_threshold=0.3)
    if result:
        print(f"✅ HIT: '{query}' -> {result['response'][:50]}...")
        print(f"   Distance: {result['vector_distance']:.3f}\n")
    else:
        print(f"❌ MISS: '{query}'\n")

### Add entries to the cache

In [ ]:
def add_to_cache(question: str, answer: str):
    """
    Add a new Q&A pair to our simple in-memory cache.
    Extends both the DataFrame and embeddings matrix.
    """
    global faq_df, faq_embeddings

    new_row = pd.DataFrame({"question": [question], "answer": [answer]})
    faq_df = pd.concat([faq_df, new_row], ignore_index=True)

    # Generate embedding for the new question
    new_embedding = encoder.encode([question])

    # Add to embeddings matrix
    faq_embeddings = np.vstack([faq_embeddings, new_embedding])

    print(f"✅ Added to cache: '{question}'")

In [ ]:
print("Original cache size:", len(faq_df))

new_entries = [
    (
        "What are your business hours?",
        "We're open Monday-Friday 9 AM to 6 PM EST. Weekend support is available for urgent issues.",
    ),
    (
        "Do you have a mobile app?",
        "Yes! Our mobile app is available on both iOS and Android. Search for 'CustomerApp' in your app store.",
    ),
    (
        "How do I update my payment method?",
        "Go to Account Settings > Payment Methods to add, edit, or remove payment options.",
    ),
]

for question, answer in new_entries:
    add_to_cache(question, answer)

print(f"\nCache now has {len(faq_df)} total entries")

In [ ]:
test_extended_queries = [
    "What time do you open?",  
    "Is there a phone app?", 
    "How can I change my payment method?",
]

for query in test_extended_queries:
    result = check_cache(query, distance_threshold=0.3)
    if result:
        print(f"✅ HIT: '{query}' -> {result['response'][:50]}...")
        print(f"   Distance: {result['vector_distance']:.3f}\n")
    else:
        print(f"❌ MISS: '{query}'\n")

## Moving to Redis

In [ ]:
import os

REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")

In [ ]:
import redis

try:
    r = redis.Redis.from_url(REDIS_URL)
    r.ping()
    print("✅ Redis is running and accessible")
except redis.ConnectionError:
    print("❌ Cannot connect to Redis")
    raise

### Using a Cache-Optimized Embedding Model (langcache-embed-v1)
https://huggingface.co/redis/langcache-embed-v1

In [ ]:
from redisvl.utils.vectorize import HFTextVectorizer
from redisvl.extensions.cache.embeddings import EmbeddingsCache

langcache_embed = HFTextVectorizer(
    model="redis/langcache-embed-v1",
    cache=EmbeddingsCache(redis_client=r, ttl=3600)
)

### Create the Redis Semantic Cache

In [ ]:
from redisvl.extensions.cache.llm import SemanticCache

cache = SemanticCache(
    name="faq-cache",
    vectorizer=langcache_embed,
    redis_client=r,
    distance_threshold=0.3
)

### Load the Cache with FAQ Data

In [ ]:
for i in range(len(faq_df)):
    cache.store(
        prompt=faq_df.iloc[i]["question"],
        response=faq_df.iloc[i]["answer"]
    )

In [ ]:
result = cache.check("I need a refund for my purchase")

In [ ]:
result

### Implement TTL (time-to-live) policy to keep cache fresh

In [ ]:
cache.set_ttl(86400)

## End-to-End LLM Example

In [ ]:
from cache.config import load_openai_key
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

load_openai_key()

MODEL_NAME = "gpt-4o-mini"

llm = ChatOpenAI(
    model=MODEL_NAME,
    temperature=0.1,
    max_tokens=150,
)

In [ ]:
def get_llm_response(question: str) -> str:
    prompt = f"""
    You are a helpful customer support assistant. Answer this customer question concisely and professionally:
    
    Question: {question}
    
    Provide a helpful response in 1-2 sentences. If you don't have specific information, give a general helpful response.
    """
    response = llm.invoke([HumanMessage(content=prompt)])
    return response.content.strip()

In [ ]:
from cache.evals import PerfEval

perf_eval = PerfEval()

test_questions = [
    "How can I get my money back?",
    "I want a refund please",
    "What's your return policy?",
    "I forgot my password",
    "Can you help me reset my password?",
    "What are your shipping costs?",
    "Do you offer installation services?",
    "Can I schedule a phone call with support?",
    "How do I cancel my subscription?",
    "How much does shipping cost?",
    "I need to cancel my account",
]

perf_eval.set_total_queries(len(test_questions))

In [ ]:
with perf_eval:
    for i, question in enumerate(test_questions, 1):
        print(f"\n[{i}] Question: '{question}'")

        perf_eval.start()

        if cached_result := cache.check(question):
            # Cache HIT
            perf_eval.tick("cache_hit")
            print(
                f"    ✅ CACHE HIT (distance: {cached_result[0]['vector_distance']:.3f})"
            )
            print(f"    📋 Cached question: {cached_result[0]['prompt'][:80]}...")
            print(f"    📋 Cached response: {cached_result[0]['response'][:80]}...")
        else:
            # Cache MISS - call LLM
            perf_eval.tick("cache_miss")  # Time for cache check
            print(f"    ❌ CACHE MISS")
            print(f"    🤖 Calling LLM... ", end="")

            # Call LLM and track the call
            perf_eval.start()
            llm_response = get_llm_response(question)
            perf_eval.tick("llm_call")
            perf_eval.record_llm_call(MODEL_NAME, question, llm_response)
            print(f"    💬 LLM response: {llm_response[:80]}...")
            cache.store(prompt=question, response=llm_response)

In [ ]:
np.mean(perf_eval.durations_by_label['cache_hit'])

In [ ]:
np.mean(perf_eval.durations_by_label['llm_call'])

<b>Note:</b> In the above experiment we measure the latency of the cache response and a mocked latency of an LLM call. The mocked LLM call is a dummy function that sleeps for a random amount of time. The randomness in the results mainly comes from the randomness we introduced to mock the LLM. The results show us what we can typically see in practice.

In [ ]:
cache.clear()